In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Activation, Flatten
print("Imported model related libs")

Using TensorFlow backend.


Imported model related libs


In [2]:
import gensim
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import warnings
import re  #regular expression for data pre processing
from collections import defaultdict
import spacy

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, conll2000
print("Imported word embedding/misc libs")

Imported word embedding/misc libs


In [3]:
import os
from os import listdir
from os.path import isfile, join
mypath = "C:/Users/varun/Desktop/Work/reviews/train"
pos_revs = []
neg_revs = []

def get_review_info(file_path):
    onlyfiles = []
    for f in listdir(file_path):
        if os.path.isfile(os.path.join(file_path, f)):
            onlyfiles.append(f)
    return onlyfiles
            
pos_revs = get_review_info(mypath + "/pos")
neg_revs = get_review_info(mypath + "/neg")

assert len(neg_revs) == 12500, "ERROR: No. of negative reviews not 12500"
assert len(pos_revs) == 12500, "ERROR: No. of positive reviews not 12500"
print("success")
    
    
def load_dataset(file_loc):
    files = file_loc + "/pos"
    all_reviews = []
    tokens = []
    y_train = []
    for x in range(0,2):
        for file in listdir(files):
            f = open(files + "/" + file, encoding="utf8")
            for word in f.read().split():
                tokens.append(word)
            all_reviews.append(tokens)
            tokens = []
        files = file_loc + "/neg"
    
    print(len(all_reviews))
    assert len(all_reviews) == 25000, "ERROR: Total number of reviews does not add up to 25000..."
    return all_reviews

list = load_dataset(mypath) #list of text samples

#for item in list[:5]:
 #   print(item)

print(len(list[1]))


success
25000
428


In [4]:
embeddings_index = {}
f = open(os.path.join('', 'word2vec_embedding_imdb_dataset.txt'), encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

#print(embeddings_index["good"])

In [5]:
#Training the word2vec model
w2vmodel = Word2Vec(list, size = 128, min_count = 1,window = 3, sg = 1) #we want to use skip-gram (takes in a word, guesses
                                                                        #the words around it)
print(len(w2vmodel.wv.vocab))
filename = "word2vec_embedding_imdb_dataset.txt"
w2vmodel.wv.save_word2vec_format(filename, binary=False)

280617


In [6]:
w2vmodel.wv.most_similar('crazy')

[('freaky', 0.840491771697998),
 ('mad', 0.8108929395675659),
 ('rapping', 0.8106375932693481),
 ('demon', 0.800931453704834),
 ('deformed', 0.8001381158828735),
 ('fat,', 0.8000245094299316),
 ('bald', 0.7986127138137817),
 ('friendly', 0.7966055870056152),
 ('clown', 0.794162392616272),
 ('weird', 0.7928417325019836)]

In [7]:
w2vmodel['bad']


C:\Users\varun\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.15717083,  0.00719881, -0.13486277,  0.24161513,  0.5917264 ,
       -0.38161474,  0.0981496 ,  0.06785046, -0.13178508, -0.2610271 ,
        0.36278152, -0.74105865,  0.02209078,  0.5998116 ,  0.11493874,
       -0.36380413,  0.33248094, -0.2831874 , -0.32796293, -0.5830784 ,
       -0.2930365 , -0.06944399, -1.1124429 ,  0.6153449 ,  0.16190404,
        0.01954959, -0.08898558, -0.13666491, -0.13935591, -0.4815852 ,
        0.543503  ,  0.04405535, -0.2948441 ,  0.38509563,  0.95225906,
       -0.16870022,  0.39097536,  0.30480352,  0.80282307,  0.8503552 ,
        0.36540058,  0.86473316,  0.17450067,  0.2526917 ,  0.26038042,
        0.04001397,  0.1455762 ,  0.23201808, -0.6949306 , -0.3180783 ,
        0.33362773,  0.34399712, -0.41319567,  0.41418356,  0.13962907,
        0.25074008, -0.10245742, -0.7726181 , -0.24751253, -0.2635865 ,
        0.05062889, -0.53740066,  1.0597956 ,  0.5728781 ,  0.02128099,
       -0.09101702, -0.42029145,  0.23381889, -0.03530229,  0.36

In [8]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(list)
sequences = tokenizer_obj.texts_to_sequences(list)


print(len(sequences[1])) #it works!!!!


#pad sequences
imdb_word_index = tokenizer_obj.word_index
print(len(imdb_word_index))

padded_list = pad_sequences(sequences, maxlen=2560)

print(padded_list.shape)
y_train = []
for x in range(0,12500):
    y_train.append(1)
for x in range(0,12500):
    y_train.append(0)


428
251637
(25000, 2560)


In [9]:

num_words = len(imdb_word_index) + 1
embedding_matrix = np.zeros((num_words, 128))

for word, i in imdb_word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
print(embedding_matrix.shape)


(251638, 128)


In [10]:
#build model!!!
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

model = keras.Sequential()
model.add(Embedding(num_words, 
                    128, 
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=2560, 
                    trainable=False))

model.add(GRU(units=32, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

W0722 13:56:54.296079 16348 deprecation_wrapper.py:119] From C:\Users\varun\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 13:56:54.376868 16348 deprecation_wrapper.py:119] From C:\Users\varun\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 13:56:56.019471 16348 deprecation_wrapper.py:119] From C:\Users\varun\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 13:56:56.135198 16348 deprecation_wrapper.py:119] From C:\Users\varun\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 13:56:56.145138 16348 deprecation.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2560, 128)         32209664  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                15456     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 32,225,153
Trainable params: 15,489
Non-trainable params: 32,209,664
_________________________________________________________________


In [11]:
model.fit(padded_list, y_train, batch_size=256, epochs=5, verbose=1)
model.save("my_W2V_model.h5")


Epoch 1/5
25000/25000 [==============================] - ETA: 40:03 - loss: 0.7031 - acc: 0.51 - ETA: 26:54 - loss: 0.7075 - acc: 0.49 - ETA: 22:10 - loss: 0.7028 - acc: 0.48 - ETA: 19:23 - loss: 0.7004 - acc: 0.50 - ETA: 17:38 - loss: 0.7014 - acc: 0.49 - ETA: 16:17 - loss: 0.7005 - acc: 0.49 - ETA: 15:28 - loss: 0.6999 - acc: 0.50 - ETA: 14:47 - loss: 0.7003 - acc: 0.49 - ETA: 14:09 - loss: 0.6996 - acc: 0.50 - ETA: 13:36 - loss: 0.6993 - acc: 0.49 - ETA: 13:07 - loss: 0.6983 - acc: 0.50 - ETA: 12:47 - loss: 0.6975 - acc: 0.50 - ETA: 12:33 - loss: 0.6974 - acc: 0.50 - ETA: 12:14 - loss: 0.6965 - acc: 0.50 - ETA: 11:55 - loss: 0.6966 - acc: 0.50 - ETA: 11:38 - loss: 0.6957 - acc: 0.50 - ETA: 11:24 - loss: 0.6956 - acc: 0.50 - ETA: 11:10 - loss: 0.6952 - acc: 0.51 - ETA: 10:56 - loss: 0.6950 - acc: 0.51 - ETA: 10:42 - loss: 0.6950 - acc: 0.51 - ETA: 10:28 - loss: 0.6951 - acc: 0.51 - ETA: 10:16 - loss: 0.6947 - acc: 0.51 - ETA: 10:04 - loss: 0.6947 - acc: 0.51 - ETA: 9:53 - loss: 0.694

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model 
model = load_model('my_W2V_model.h5')

import keras


sentence = input("Type in sentence.")
type(sentence)

tokens = word_tokenize(sentence)
print(tokens)

testIntArray = []

for word in tokens:
    if word in imdb_word_index:
        testIntArray.append(imdb_word_index[word])
    

finalTestArray = np.array([testIntArray])
finalTestArray = keras.preprocessing.sequence.pad_sequences(finalTestArray, maxlen=256, value=0, padding='post')
print(finalTestArray)

model.predict(finalTestArray)